In [35]:
!pip show pinder

Name: pinder
Version: 0.5.0
Summary: PINDER: The Protein INteraction Dataset and Evaluation Resource
Home-page: https://github.com/pinder-org/pinder
Author: 
Author-email: 
License: 
Location: /home/a03-svincoff/miniconda3/envs/interact/lib/python3.10/site-packages
Requires: biotite, fastpdb, gcsfs, google-cloud-storage, nbformat, numpy, pandas, plotly, pyarrow, pydantic, PyYAML, scikit-learn, tabulate, torch, torch-geometric, torchtyping, tqdm, typeguard, typing-extensions
Required-by: 


In [36]:
from pinder.core import get_metadata
import numpy as np

metadata = get_metadata()
metadata["resolution"] = metadata["resolution"].astype(float)
metadata = metadata.replace("", np.nan)
metadata

/tmp/ipykernel_2464472/1003134201.py:6: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  metadata = metadata.replace("", np.nan)


,id,entry_id,method,date,release_date,resolution,label,probability,chain1_id,chain2_id,...,interface_atom_gaps_4A,missing_interface_residues_4A,interface_atom_gaps_8A,missing_interface_residues_8A,entity_id_R,entity_id_L,pdb_strand_id_R,pdb_strand_id_L,ECOD_names_R,ECOD_names_L
0,7rzb__A1_A0A229LVN5--7rzb__A2_A0A229LVN5,7rzb,X-RAY DIFFRACTION,2021-08-27,2022-04-13,1.599609,BIO,0.576172,R,L,...,0,0,0,0,1,1,A,A,PF06491,PF06491
1,3t2l__A1_Q5LE95--3t2l__A2_Q5LE95,3t2l,X-RAY DIFFRACTION,2011-07-22,2011-08-10,2.330078,BIO,0.983887,R,L,...,0,0,0,0,1,1,A,A,"F_UNCLASSIFIED,PF13149","F_UNCLASSIFIED,PF13149"
2,6ikj__A1_Q9I4L6--6ikj__B1_Q9I4L6,6ikj,X-RAY DIFFRACTION,2018-10-16,2019-03-13,1.759766,BIO,0.543945,R,L,...,0,0,0,0,1,1,A,B,PF00691,PF00691
3,8iyi__A1_Q6CVU4--8iyi__B1_Q6CVU4,8iyi,X-RAY DIFFRACTION,2023-04-05,2023-06-28,1.900391,BIO,0.992188,R,L,...,0,0,0,0,1,1,A,B,"PF17284,PF01564","PF17284,PF01564"
4,3uws__B1_A7A9N3--3uws__A1_A7A9N3,3uws,X-RAY DIFFRACTION,2011-12-02,2012-06-13,1.700195,BIO,0.996094,R,L,...,0,0,0,0,2,1,B,A,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2319559,6hbg__C28_Q8V635--6hbg__C35_Q8V635,6hbg,ELECTRON MICROSCOPY,2018-08-10,2019-03-20,3.160156,BIO,0.512207,R,L,...,0,0,0,0,3,3,C,C,PF00073,PF00073
2319560,6hbg__A11_Q8V635--6hbg__A54_Q8V635,6hbg,ELECTRON MICROSCOPY,2018-08-10,2019-03-20,3.160156,NaN,0.000000,R,L,...,0,0,0,0,1,1,A,A,PF00073,PF00073
2319561,6hbg__C15_Q8V635--6hbg__D4_Q8V635,6hbg,ELECTRON MICROSCOPY,2018-08-10,2019-03-20,3.160156,XTAL,0.491943,R,L,...,0,0,0,0,3,4,C,D,PF00073,PF02226
2319562,6hbg__C33_Q8V635--6hbg__D52_Q8V635,6hbg,ELECTRON MICROSCOPY,2018-08-10,2019-03-20,3.160156,XTAL,0.491943,R,L,...,0,0,0,0,3,4,C,D,PF00073,PF02226


In [121]:
metadata["long_key"] = metadata["entry_id"].astype(str)+"_"+metadata["chain1_id"].astype(str)+"_"+metadata["chain2_id"].astype(str)+"_"+metadata["asym_id_1"].astype(str)+"_"+metadata["asym_id_2"].astype(str)
duplicate_rows = metadata.loc[metadata.duplicated("long_key")]["long_key"].unique()
metadata.loc[
    metadata["long_key"].isin(duplicate_rows)
    ].sort_values(by=["long_key"]).reset_index(drop=True).iloc[0:10,:].to_csv("example_dups_pinder.csv", index=False)

In [37]:
metadata["label"].value_counts(dropna=False)

label
BIO     1180362
XTAL     873090
NaN      266112
Name: count, dtype: int64

In [123]:
negatives = metadata.loc[
    (metadata["label"].isna()) & 
    (metadata["resolution"]<=3.5) 
].reset_index(drop=True)
print(f"total negatives: {len(negatives):,}")
total_negs_no_filter = len(metadata.loc[metadata['label'].isna()])
print(f"total negatives without resolution req: {total_negs_no_filter:,}")
display(negatives.head())
display(negatives["probability"].value_counts(dropna=False))

total negatives: 152,551
total negatives without resolution req: 266,112


,id,entry_id,method,date,release_date,resolution,label,probability,chain1_id,chain2_id,...,missing_interface_residues_4A,interface_atom_gaps_8A,missing_interface_residues_8A,entity_id_R,entity_id_L,pdb_strand_id_R,pdb_strand_id_L,ECOD_names_R,ECOD_names_L,long_key
0,6fyy__SA1_P32497--6fyy__EA1_Q6CNL2,6fyy,ELECTRON MICROSCOPY,2018-03-12,2018-12-05,3.050781,NaN,0.0,R,L,...,0,0,0,45,31,q,b,NaN,PF01667,6fyy_R_L_SA_EA
1,7mbv__B1_S5UH55--7mbv__D1_S5UH55,7mbv,ELECTRON MICROSCOPY,2021-04-01,2021-07-07,2.800781,NaN,0.0,R,L,...,0,0,0,1,1,B,D,NaN,NaN,7mbv_R_L_B_D
2,7mbv__A1_S5UH55--7mbv__C1_S5UH55,7mbv,ELECTRON MICROSCOPY,2021-04-01,2021-07-07,2.800781,NaN,0.0,R,L,...,0,0,0,1,1,A,C,NaN,NaN,7mbv_R_L_A_C
3,5fci__IA1_P39015--5fci__Q1_Q01855,5fci,X-RAY DIFFRACTION,2015-12-15,2016-05-11,3.400391,NaN,0.0,R,L,...,0,0,0,35,17,SM,C5,F_UNCLASSIFIED,PF00203,5fci_R_L_IA_Q
4,5fci__VA1_P0C0W9--5fci__IA1_P39015,5fci,X-RAY DIFFRACTION,2015-12-15,2016-05-11,3.400391,NaN,0.0,R,L,...,0,0,0,48,35,M1,SM,"PF00281,PF00673",F_UNCLASSIFIED,5fci_R_L_VA_IA


probability
0.0    152551
Name: count, dtype: int64

In [124]:
negatives["long_key"] = negatives["entry_id"].astype(str)+"_"+negatives["chain1_id"].astype(str)+"_"+negatives["chain2_id"].astype(str)+"_"+negatives["asym_id_1"].astype(str)+"_"+negatives["asym_id_2"].astype(str)
duplicate_rows = negatives.loc[negatives.duplicated("long_key")]["long_key"].unique()
negatives.loc[
    negatives["long_key"].isin(duplicate_rows)
    ].sort_values(by=["long_key"]).reset_index(drop=True).iloc[0:10,:].to_csv("example_dups_pinder_negs.csv", index=False)

In [67]:
vc = negatives["asym_id_1"].value_counts().reset_index()
vc.loc[vc["count"]>0]
negatives.loc[negatives["asym_id_1"]=="GO"][["entry_id", "chain1_id","chain2_id","asym_id_1","asym_id_2", "length1", "length2", "length_resolved_1", "length_resolved_2"]]

,entry_id,chain1_id,chain2_id,asym_id_1,asym_id_2,length1,length2,length_resolved_1,length_resolved_2
47134,8snb,R,L,GO,S,437,101,437,101


In [127]:
print(len(metadata))
print(len(metadata["id"].unique()))

2319564
2319564


In [58]:
negatives.loc[
    negatives["length1"]!=negatives["length_resolved_1"]
    ][["entry_id", "chain1_id","chain2_id","asym_id_1","asym_id_2", "length1", "length2", "length_resolved_1", "length_resolved_2"]]

,entry_id,chain1_id,chain2_id,asym_id_1,asym_id_2,length1,length2,length_resolved_1,length_resolved_2
1689,7zwn,R,L,B,C,6,6,5,5
1832,2adw,R,L,G,H,8,8,10,8
2060,1f8v,R,L,C,G,26,22,25,21
2365,3q9g,R,L,A,A,9,9,10,10
2383,3q9i,R,L,B,D,9,9,10,10
...,...,...,...,...,...,...,...,...,...
48335,2zs1,R,L,D,A,145,140,155,140
48336,2zs0,R,L,D,A,145,140,155,140
48337,2zs0,R,L,D,B,145,142,155,142
48433,4k0k,R,L,Q,O,100,88,99,88


In [25]:
from protparser.rcsb import download_rcsb
from Bio.PDB.MMCIFParser import MMCIFParser

# test downloading a structure and make sure it can be parsed
pdb_id = "7QFT"
output_dir = "downloads"
cifparser = MMCIFParser()
download_rcsb(pdb_id, struct_format="cif", output_dir=None)

# try parsing
structure = cifparser.get_structure(pdb_id, f'{output_dir}/{pdb_id}.cif')
print("parsed!")

parsed!


/home/a03-svincoff/miniconda3/envs/interact/lib/python3.10/site-packages/Bio/PDB/StructureBuilder.py:100: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 3486.
  warnings.warn(
/home/a03-svincoff/miniconda3/envs/interact/lib/python3.10/site-packages/Bio/PDB/StructureBuilder.py:100: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 3704.
  warnings.warn(


In [101]:
from protparser.rcsb import RCSBStructure

def get_chain_sequences(pdb_id, chain1, chain2):
    rcsbstructure = RCSBStructure(pdb_id, download_struct=False)
    chaindf = rcsbstructure.get_chain_info_df()
    chaindf["Chain"] = chaindf["Chain"].apply(lambda x: x.split(","))
    chaindf["Auth"] = chaindf["Auth"].apply(lambda x: x.split(","))
    chaindf = chaindf.explode(["Chain","Auth"])
    
    d = {}
    d["entry_id"] = pdb_id
    d["asym_id_1"] = chain1
    d["asym_id_2"] = chain2
    
    chain1_info = chaindf.loc[chaindf["Chain"]==chain1]
    chain2_info = chaindf.loc[chaindf["Chain"]==chain2]
    d["uniprot_1"] =chain1_info["UniProt IDs"].item()
    d["uniprot_2"] =chain2_info["UniProt IDs"].item()
    d["pdb_seq_1"] =chain1_info["PDB Seq"].item()
    d["pdb_seq_2"] =chain2_info["PDB Seq"].item()

    return d
        

In [104]:
import pandas as pd
neg_subset = negatives.sample(2,random_state=42)
d= neg_subset.apply(lambda row: get_chain_sequences(row["entry_id"],row["asym_id_1"], row["asym_id_2"]), axis=1)
pd.DataFrame(d.tolist())

,entry_id,asym_id_1,asym_id_2,uniprot_1,uniprot_2,pdb_seq_1,pdb_seq_2
0,6x6s,UD,DE,A0A2J9KJK3,A0A2J9KJK3,MFRKLATAVSLIGLLTSNTLYAKEISEADKVIKATKETKETKKEAK...,MFRKLATAVSLIGLLTSNTLYAKEISEADKVIKATKETKETKKEAK...
1,6tdz,D,H,None,None,MPGGGTIRFWREKLEGYKKYHQIVKTIKMVTLAKYRQTVVRTRVRD...,MQRGSSITKVVRRAALARSTRNAAIAYEVTVNGANLIGAGMAASGV...


In [103]:
pd.DataFrame(d.tolist())

,entry_id,asym_id_1,asym_id_2,uniprot_1,uniprot_2,pdb_seq_1,pdb_seq_2
0,6x6s,UD,DE,A0A2J9KJK3,A0A2J9KJK3,MFRKLATAVSLIGLLTSNTLYAKEISEADKVIKATKETKETKKEAK...,MFRKLATAVSLIGLLTSNTLYAKEISEADKVIKATKETKETKKEAK...
1,6tdz,D,H,None,None,MPGGGTIRFWREKLEGYKKYHQIVKTIKMVTLAKYRQTVVRTRVRD...,MQRGSSITKVVRRAALARSTRNAAIAYEVTVNGANLIGAGMAASGV...


In [125]:
rcsbstructure = RCSBStructure("1a14", download_struct=False)
fasta_content= rcsbstructure.fasta_content
print(fasta_content)
chaindf = rcsbstructure.get_chain_info_df()
chaindf["Chain"] = chaindf["Chain"].apply(lambda x: x.split(","))
chaindf["Auth"] = chaindf["Auth"].apply(lambda x: x.split(","))
chaindf = chaindf.explode(["Chain","Auth"])
chaindf

>1A14_1|Chain A[auth N]|NEURAMINIDASE|Influenza A virus (11320)
RDFNNLTKGLCTINSWHIYGKDNAVRIGEDSDVLVTREPYVSCDPDECRFYALSQGTTIRGKHSNGTIHDRSQYRALISWPLSSPPTVYNSRVECIGWSSTSCHDGKTRMSICISGPNNNASAVIWYNRRPVTEINTWARNILRTQESECVCHNGVCPVVFTDGSATGPAETRIYYFKEGKILKWEPLAGTAKHIEECSCYGERAEITCTCRDNWQGSNRPVIRIDPVAMTHTSQYICSPVLTDNPRPNDPTVGKCNDPYPGNNNNGVKGFSYLDGVNTWLGRTISIASRSGYEMLKVPNALTDDKSKPTQGQTIVLNTDWSGYSGSFMDYWAEGECYRACFYVELIRGRPKEDKVWWTSNSIVSMCSSTEFLGQWDWPDGAKIEYFL
>1A14_2|Chain B[auth H]|NC10 FV (HEAVY CHAIN)|Mus musculus (10090)
QVQLQQSGAELVKPGASVRMSCKASGYTFTNYNMYWVKQSPGQGLEWIGIFYPGNGDTSYNQKFKDKATLTADKSSNTAYMQLSSLTSEDSAVYYCARSGGSYRYDGGFDYWGQGTTVTV
>1A14_3|Chain C[auth L]|NC10 FV (LIGHT CHAIN)|Mus musculus (10090)
DIELTQTTSSLSASLGDRVTISCRASQDISNYLNWYQQNPDGTVKLLIYYTSNLHSEVPSRFSGSGSGTDYSLTISNLEQEDIATYFCQQDFTLPFTFGGGTAA



,Chain,Auth,UniProt IDs,PDB Seq length,PDB Seq,Canonical Seq length,Canonical Seq,Pfam domains,Pfam domains number
0,A,N,P03472,388,RDFNNLTKGLCTINSWHIYGKDNAVRIGEDSDVLVTREPYVSCDPD...,388,RDFNNLTKGLCTINSWHIYGKDNAVRIGEDSDVLVTREPYVSCDPD...,"{'pfam_1': {'feature_id': 'PF00064', 'name': '...",1
1,B,H,P01749,120,QVQLQQSGAELVKPGASVRMSCKASGYTFTNYNMYWVKQSPGQGLE...,120,QVQLQQSGAELVKPGASVRMSCKASGYTFTNYNMYWVKQSPGQGLE...,"{'pfam_1': {'feature_id': 'PF07686', 'name': '...",1
2,C,L,P01645,104,DIELTQTTSSLSASLGDRVTISCRASQDISNYLNWYQQNPDGTVKL...,104,DIELTQTTSSLSASLGDRVTISCRASQDISNYLNWYQQNPDGTVKL...,"{'pfam_1': {'feature_id': 'PF07686', 'name': '...",1
